<a href="https://colab.research.google.com/github/SebastienBienfait/L2C-Data-managment/blob/main/Data_Roles_job_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Title

Importing libaries and defining base jobsite urls

In [46]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'  ##<- this is really annoying, I WANT to copy the DF.
import numpy as np
import datetime
import matplotlib.pyplot as plt
from google.colab import files

user input:    
To use this programe, input your search terms in the box below. 

Search Term: You can include commers, spaces and multiple words.

Center of Search, use format: 'Town, County' if the center you are searching for is not unique. If it is unique within the UK, 'County' is unnecessary. Or you can just use 'County' as your search center.

--NB: for JobsInKent, the search is limited to 'Kent' by default.

Radius: enter any valid integer. The closest valid radius for each site will be picked. If you are equidistance between two values, the lower value will be choesen.

Filter Term: This will filter the final database to only contain jobs with the Filiter Term in the title: ie 'Data' will filter keep 'Data Developer' but not 'Software Developer'. To not filter at all, type 'no' instead.

--NB: the filter term is case sensitve, 'data' will not pick up 'Database'

In [47]:
def user_imput():
  search_term = input("Search Term: " )
  search_center = input("Center of Search: ") #town, county
  search_radius = int(input("Search Radius: " ))

  filter_term = input("Filter Term: ")
  if filter_term.lower() == "no":
    filter_term = None
  return search_term, search_center, search_radius, filter_term



search_term, search_center, search_radius, filter_term = user_imput()


Search Term: Data
Center of Search: Kent
Search Radius: 10
Filter Term: Data


In [48]:
def find_valid_radius(radius,valid_list):
  valid_test = np.array([],dtype="int")

  for radi in valid_list:
    valid_test = np.append(valid_test,[abs(radi-radius)])
  return_radius = valid_list[np.argmin(valid_test)]
  return return_radius

In [49]:
def creat_urls(search_term, search_center, search_radius):
  jobsinkent_search_str = search_term.replace(",","%2C").replace(" ","+")
  reed_search_str = search_term.replace(",","%2C").replace(" ","%20")

  reed_center_str = search_center.replace(", ","%2C-")


  jobsinkent_valid_radius = np.array([1,2,5,10,20,30,40],dtype="int")
  jobsinkent_radius = find_valid_radius(search_radius, jobsinkent_valid_radius)

  reed_valid_radius = np.array([0,1,3,5,10,15,20,30,50],dtype="int")
  reed_radius = find_valid_radius(search_radius, reed_valid_radius)


  jobsinkent_url = "https://jobsinkent.com/search?q=" + jobsinkent_search_str + "&pl=1" + "&r=" + str(jobsinkent_radius)
  reed_url = "https://www.reed.co.uk/jobs?keywords=" + reed_search_str+"&location=" + reed_center_str + "&proximity=" + str(reed_radius)
  print(jobsinkent_url)
  print(reed_url)


  save_state = False
  query_save = input("Would you like to save and download the results of your search? (yes/no) ")
  if query_save.lower() == "yes":
    save_state = True
  else: 
    save_state = False

  return reed_url, jobsinkent_url, save_state

In [50]:
reed_url, jobsinkent_url, save_state = creat_urls(search_term, search_center, search_radius)

https://jobsinkent.com/search?q=Data&pl=1&r=10
https://www.reed.co.uk/jobs?keywords=Data&location=Kent&proximity=10
Would you like to save and download the results of your search? (yes/no) no


In [51]:
#reed_url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"
#jobsinkent_url = "https://jobsinkent.com/search?q=Data&pl=1" #pl=1 -> Kent

max_job_call = 2000 #reed has a limit of 2000 job search api calls per hour.

General Functions that are used in both searches.

In [61]:
#returns the full HTML of a page.
def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

#This cleans the dataframes when they have been created.
def general_clean(df,site,filter_term):
  df.drop_duplicates(inplace=True)

  if filter_term != None:
    df = df[df["job_title"].str.contains(filter_term)]


  df["salary"].replace(np.nan, '', regex=False,inplace=True) #not all companies have listed a salary and so NaN is sometimes returned
  df["company"] = df["company"].str.lstrip(" ").str.rstrip(" ") #some companies have excess whitespaces around their names

  df["date_found"] = datetime.date.today()
  df["advertised_on"] = site
  return df

#Reed Job Search
---

This function finds the job ID for all jobs displayed on each page url from reed.co.uk


In [54]:
def reed_jobs_iterate(page_url):
  site_html = get_html(page_url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  job_cards = results.find_all(class_="job-result-card")

  for job in job_cards: #finds the jobs, gets it's ID, returns the id
      job_id = int(job["id"].split("jobSection")[1]) #jobSection48529572 ect...
      
      id_list = np.append(id_list,[job_id])
  return id_list

This finds the total jobs found in the search

In [55]:
def find_reed_total_jobs(site_html,max_job_call):
  max_page_requests = int(max_job_call/25) 

  ###
  total_jobs_text = site_html.find(class_="col-sm-11 col-xs-12 page-title").text # '\n' '\r' '\n' x,xxx\r\n  Data Jobs near Ashford       '\n'....
  total_jobs_text = total_jobs_text.replace("\n","").replace("\r","").replace(",","") #          xxxx         Data Jobs near Ashford       .       

  total_jobs = int(total_jobs_text.split("Data")[0].strip(" "))
  total_pages = int(np.ceil(total_jobs/25))

  #limits the requests to only 2000
  if total_pages > max_page_requests:
    total_pages = max_page_requests


  return total_jobs, total_pages


The function srapes website for all job id's that meet the search critiera.


In [56]:
def reed_scrape(base_url,max_job_call):
  soup = get_html(base_url)
  total_id_list = np.array([],dtype=int)

  #finds out how many more pages to scrape
  total_jobs, total_pages = find_reed_total_jobs(soup,max_job_call)

  print("total jobs found: ", total_jobs)
  print("total pages: ", total_pages)
  print("pages to search: ",total_pages)

  for page_no in np.arange(1,total_pages+1): 
    page_url = base_url+"&pageno="+str(page_no)
    page_id_list = reed_jobs_iterate(page_url)
    total_id_list = np.append(total_id_list,[page_id_list])

    #user feedback
    print("page ",page_no, " processed")
  return total_id_list


This calls the reed API to return job information based on the jobID

In [57]:
def call_reed_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  api_url = ""
  job_df = pd.DataFrame()

  #user feedback
  print("Retriving job information...")

  for job_id in id_list:
    api_url = base_url+str(job_id)
    post_request = requests.get(api_url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
    json_data = post_request.json()

    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df

Calls the scraping function and the api function in turn ~7-8mins to run

In [ ]:
reed_id_list = reed_scrape(reed_url,max_job_call)
reed_id_list = reed_id_list.astype("int") #it really didnt want to save the array as an int

full_reed_df = call_reed_api(reed_id_list)
display(full_reed_df.info())

Cleaning the Reed DF

In [82]:
def general_clean(df,site,filter_term):
  df.drop_duplicates(inplace=True)
  df["salary"].replace(np.nan, '', regex=False,inplace=True) #not all companies have listed a salary and so NaN is sometimes returned
  df["company"] = df["company"].str.lstrip(" ").str.rstrip(" ") #some companies have excess whitespaces around their names
  df.dropna(inplace=True) #just incase the max of 2000 jobs is bypassed

  if filter_term != None:
    df = df[df["job_title"].str.contains(filter_term)]

  df["date_found"] = datetime.date.today()
  df["advertised_on"] = site
  return df

In [84]:
def clean_reed(df_in): #this filters out the necessary rows and renames them so the cleaning function works.
  df = df_in[["jobTitle", "employerName", "salary", "contractType", "locationName",  "jobUrl"]]

  df.rename(columns={"jobTitle":"job_title", "employerName": "company", "contractType":"contract", "locationName":"location",  "jobUrl":"job_url"}, inplace=True)
  return df

reed_df = clean_reed(full_reed_df)
reed_df = general_clean(reed_df,"https://www.reed.co.uk",filter_term)
display(reed_df.shape)
display(reed_df.head(5))

(1201, 8)

,job_title,company,salary,contract,location,job_url,date_found,advertised_on
0,Data Architect,The Rank Group,,Permanent,London,https://www.reed.co.uk/jobs/data-architect/484...,2022-11-01,https://www.reed.co.uk
1,Data Architect,Veolia UK,"£65,000 - £84,600 per annum, inc benefits",Permanent,United Kingdom,https://www.reed.co.uk/jobs/data-architect/486...,2022-11-01,https://www.reed.co.uk
2,Data Inputter,Clemence Rogers Recruitment,"£20,000 per annum",Permanent,Belvedere,https://www.reed.co.uk/jobs/data-inputter/4850...,2022-11-01,https://www.reed.co.uk
3,Data Analyst,Morgan Sindall Property Services,"£35,000 - £40,000 per annum, inc benefits",Permanent,Dartford,https://www.reed.co.uk/jobs/data-analyst/48211407,2022-11-01,https://www.reed.co.uk
5,Data Administrator,Braundton Consulting Limited,"£22,194 per annum",Permanent,Sidcup,https://www.reed.co.uk/jobs/data-administrator...,2022-11-01,https://www.reed.co.uk


#JobsInKent Job Search
---


This function returns a DF containing all of the relevent jobs found on 1 page

In [85]:
def jobsinkent_iterate(url,job_limit):
  jobs_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])
  site_html = get_html(url)
  results = site_html.find(class_="mt-2 lg:mt-0")

  #this is finding an additional 20 jobs per page that are not visible on the site itself.
  job_cards = results.find_all(class_="flex mb-2 border-black-900 rounded-md border-x-4 sm:rounded-lg")


  #[:job_limit] is required as non relevent jobs were being found in the site_html with the job_cards class after the 20~ jobs that were being show on the site.
  # So I had to find how many jobs were being show in each page first and then limit the loop to only iterate up to that number of jobs.
  for job in job_cards[:job_limit]:

    job_title_url = job.find("a", class_="text-blue-700 visited:text-grey-200 hover:underline") 
    job_title = job_title_url.text
    job_url = job_title_url["href"]

    company_name = job.find(class_="mt-0 mb-2 max-w-2xl text-sm text-black").text 

    #This info was all on the fouth box class and needed to be seperated further for the relevent information to be extracted.
    info_list = job.find_all(class_="col-span-12") 
    salary = clean_bloat(info_list[0].text)
    contract = clean_bloat(info_list[1].text)
    location = clean_bloat(info_list[2].text)

    new_row = {"job_title":job_title, "company":company_name, "salary":salary, "contract":contract, "location":location,"job_url":job_url}
    jobs_df = jobs_df.append(new_row,ignore_index = True)
  return jobs_df

In [86]:
#removes '\n' and spaces from from names.
def clean_bloat(text):
  strip1 = text.replace("\n","").replace("\xa0","")
  strip2 = strip1.strip(" ")
  return strip2

#extracts information about numbers of jobs 
def return_jobs_pages(input_str):
  p_j = input_str.split("of") #[showing 1 to yy] , [xx jobs]
  jobs_per_page = int(p_j[0].split("to")[1]) #[yy]
  total_jobs = int(p_j[1].split("Jobs")[0]) #[xx]
  
  total_pages = int(np.ceil(total_jobs/jobs_per_page))

  return total_jobs, jobs_per_page, total_pages

The main function that calls to find out how many pages are needed to search over and stores the results of each iteration call.

In [87]:
def jobsinkent_scrape(base_url):
  #initialse the DF that will be used
  jobsinkent_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])

  #returns ALL the html from the site
  site_html = get_html(base_url)

  banner = site_html.find("p", class_="text-sm text-gray-700 leading-5") #~6th child within the top banner.
  showing = banner.text # 'Showing 1 to yy of xx Jobs'

  total_jobs, jobs_per_page, total_pages = return_jobs_pages(showing)

  for page_no in range(1,total_pages+1):
    #this if statment is required due to the issues mentioned above about job limits. It sets the jobs_limit variable to however many jobs there are displayed (visibly) on the page.
    if page_no == total_pages:
      jobs_limit = total_jobs%jobs_per_page
    else:
      jobs_limit = jobs_per_page

    page_url = base_url + "&page=" + str(page_no)
    new_df = jobsinkent_iterate(page_url, jobs_limit)
    jobsinkent_df = jobsinkent_df.append(new_df, ignore_index=True)
  
    #user feedback  
    print("page ",page_no, " processed")

  return jobsinkent_df
  
jobsinkent_df = jobsinkent_scrape(jobsinkent_url)
display(jobsinkent_df.head(5))

page  1  processed
page  2  processed
page  3  processed


,job_title,company,salary,contract,location,job_url
0,Invoice Clerk / Data Entry,Huntress,Up to £9.80 per hour + Holiday pay + Free parking,"temporary,full-time","Dartford, Kent",https://jobsinkent.com/job/2047926
1,Database Administrator,Pearson Whiffin Recruitment,£20000 - £23000 per annum,"permanent,full-time","Sittingbourne, Kent",https://jobsinkent.com/job/2282433
2,Senior Data Analyst,Clarion Housing Group Ltd,£43264 - £59488 per annum,"permanent,full-time","Central London, Greater London",https://jobsinkent.com/job/2423530
3,Data Analyst - Global Insurance Markets,MW Appointments,Negotiable,"permanent,full-time","City of London, Greater London",https://jobsinkent.com/job/2422053
4,Data Analyst,Pearson Whiffin Recruitment,£23000 - £28000 per annum + DOE,"permanent,full-time","West Malling, Kent",https://jobsinkent.com/job/2331144


Cleans the jobsinkent_df to be used later.

In [89]:
jobsinkent_df = general_clean(jobsinkent_df,"jobsinkent.com",filter_term)

#Combining both results
---


Combines the two DF's from JobsInKent

In [90]:
data_jobs_df = pd.DataFrame()

data_jobs_df = data_jobs_df.append([jobsinkent_df,reed_df],ignore_index=True)
display(data_jobs_df)

,job_title,company,salary,contract,location,job_url,date_found,advertised_on
0,Invoice Clerk / Data Entry,Huntress,Up to £9.80 per hour + Holiday pay + Free parking,"temporary,full-time","Dartford, Kent",https://jobsinkent.com/job/2047926,2022-11-01,jobsinkent.com
1,Database Administrator,Pearson Whiffin Recruitment,£20000 - £23000 per annum,"permanent,full-time","Sittingbourne, Kent",https://jobsinkent.com/job/2282433,2022-11-01,jobsinkent.com
2,Senior Data Analyst,Clarion Housing Group Ltd,£43264 - £59488 per annum,"permanent,full-time","Central London, Greater London",https://jobsinkent.com/job/2423530,2022-11-01,jobsinkent.com
3,Data Analyst - Global Insurance Markets,MW Appointments,Negotiable,"permanent,full-time","City of London, Greater London",https://jobsinkent.com/job/2422053,2022-11-01,jobsinkent.com
4,Data Analyst,Pearson Whiffin Recruitment,£23000 - £28000 per annum + DOE,"permanent,full-time","West Malling, Kent",https://jobsinkent.com/job/2331144,2022-11-01,jobsinkent.com
...,...,...,...,...,...,...,...,...
1246,Freelance Account Director (CRM-Data Marketin...,Bodhi Resourcing,£250.00 - £300.00 per day,Permanent,London,https://www.reed.co.uk/jobs/freelance-account-...,2022-11-01,https://www.reed.co.uk
1247,Senior E-Trading Platform Engineer - Python/Da...,Harvey Nash,£850.00 - £950.00 per day,Contract,London,https://www.reed.co.uk/jobs/senior-e-trading-p...,2022-11-01,https://www.reed.co.uk
1248,Business Development Manager (Tech & Data) - L...,Leighton Taylor Consulting,"£56,000 - £65,000 per annum",Permanent,London,https://www.reed.co.uk/jobs/business-developme...,2022-11-01,https://www.reed.co.uk
1249,"Solutions Architect - AWS, Azure, GCP, DevOps,...",Opus Recruitment Solutions Ltd,"£80,000 - £100,000 per annum",Permanent,London,https://www.reed.co.uk/jobs/solutions-architec...,2022-11-01,https://www.reed.co.uk


This returns a **unique** list of recruiters and compaies who are advertiseing data roles

In [91]:
def find_company_lists():
  data_jobs_employer_names = data_jobs_df.drop_duplicates(subset = "company") #data_jobs_df["company"].unique()

  #seperating out all recruitment companies from the list
  companies_df = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==True].index)

  #This requires going through each company and deciding if they are a recruitment agency or not if they dont have 'Recruit' in their name and adding them to the list.
  additional_recruiter_list = ["Reed","OnetoOne Personnel","Huntress","MW Appointments","P3 Search & Selection","Manpower - Ashford","Morgan McKinley","Brook Street","McGregor Boyall","SAGA","Office Angels","REED",
                            "Commercial Services Interim & Executive Search","Harnham","Connect2Staff","Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","Morgan Law", "Academics","eFinancialCareers",
                            "Commercial Services HR", "OBJECTIVE HR LTD" ]
  it_specialits = ["Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","eFinancialCareers"]

  companies_df = companies_df.drop( companies_df[companies_df["company"].isin(additional_recruiter_list) == True ].index)
  companies_list = companies_df["company"]

  recruiters_df1 = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==False].index) #drop 'Recruit' is not in name
  recruiters_df2 = data_jobs_employer_names.drop( data_jobs_employer_names[data_jobs_employer_names["company"].isin(additional_recruiter_list) == False ].index) #drop if company is not in 'additional_drop_list'

  recruiters_df = recruiters_df1.append(recruiters_df2,ignore_index=True) #combine the two sets of lists
  recruiters_list = recruiters_df["company"]

  return companies_list, recruiters_list

companies_list, recruiters_list = find_company_lists()

In [92]:
display(companies_list)

2             Clarion Housing Group Ltd
7                       PW Construction
12      Commercial Services Trading Ltd
50                       The Rank Group
51                            Veolia UK
                     ...               
1235          Skillfinder International
1236                   Bodhi Resourcing
1239                          VMA Group
1241                               SS&C
1248         Leighton Taylor Consulting
Name: company, Length: 255, dtype: object

This takes the lists of unique compaies and recruiters and returns a DataFrame with **all** the roles they have advertised.

In [93]:
def find_company_df():
  company_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(recruiters_list) == True ].index) #drop if company is a recruiter
  recruiters_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(companies_list) == True ].index) #drop if company is not a recruiter

  #For readability.
  company_df.reset_index(drop=True, inplace = True)
  recruiters_df.reset_index(drop=True, inplace = True)

  return company_df, recruiters_df

company_df, recruiters_df = find_company_df()
  

To download and save the DataFrames, uncomment save() and download() and run the cell

In [94]:
#saveing files names of search based on current date.
date_today = datetime.date.today()
recruiters_fn = str(date_today)+"_recruitment_company_df.csv"
company_fn = str(date_today)+"_data_employeer_df.csv"

#creats a .csv file in colab (cloud?) NB: This does not save the .csv files to you PC, you need to download them first.
def save():
  recruiters_df.to_csv(recruiters_fn)
  company_df.to_csv(company_fn)

#downloads files to pc
def download():
  files.download(recruiters_fn)
  files.download(company_fn)

###Uncomment to save and download, commented by default.

In [96]:
if save_state == True:
  save()
  download()
  print("Saved and Printed")
else:
  print("You did not save the DF")

You did not save the DF
